<a href="https://colab.research.google.com/github/Daltonlbs/ColabProjects/blob/main/Suporte%2BResistencia_pela_Volatilidade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance
!pip install fix_yahoo_finance
!pip install pandas_datareader --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.3 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 109 kB 12.7 MB/s 
  Attempting uninstall: pandas-datareader
    Found existing installation: pandas-datareader 0.9.0
    Uninstalling pandas-datareader-0.9.0:
      Successfully uninstalled pandas-datareader-0.9.0


In [ ]:
import pandas as pd
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go


import yfinance as yf
#import fix_yahoo_finance as yf
yf.pdr_override()   # Corrigir erro: 'Error using pandas-datareader: No data fetched using 'YahooDailyReader'

get_ipython().run_line_magic("matplotlib", "inline")
import warnings
warnings.filterwarnings("ignore")

In [ ]:
ticker1 = "^GSPC"
df1 = yf.download(ticker1, "2012-01-01", "2022-12-31")
df1["Returns"] =  df1["Adj Close"].pct_change(1)
df1["Adj Low"] = df1["Low"] - (df1["Close"]-df1["Adj Close"])
df1["Adj High"] = df1["High"] - (df1["Close"]-df1["Adj Close"])
df1["Adj Open"] = df1["Open"] - (df1["Close"]-df1["Adj Close"])
df1["Target"] = df1["Returns"].shift(-1)
df1.tail()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,Returns,Adj Low,Adj High,Adj Open,Target
Date,,,,,,,,,,,
2022-11-14,3977.969971,4008.969971,3956.399902,3957.250000,3957.250000,4561930000,-0.008936,3956.399902,4008.969971,3977.969971,0.008713
2022-11-15,4006.409912,4028.840088,3953.169922,3991.729980,3991.729980,5015310000,0.008713,3953.169922,4028.840088,4006.409912,-0.008252
2022-11-16,3976.820068,3983.090088,3954.340088,3958.790039,3958.790039,4165320000,-0.008252,3954.340088,3983.090088,3976.820068,-0.003089
2022-11-17,3919.260010,3954.330078,3906.540039,3946.560059,3946.560059,4051780000,-0.003089,3906.540039,3954.330078,3919.260010,0.004759
2022-11-18,3966.389893,3979.889893,3935.979980,3965.340088,3965.340088,4037360000,0.004759,3935.979980,3979.889893,3966.389893,NaN


In [ ]:
vol_p1 = 20
df1["Vol"] = np.round(df1["Returns"].rolling(vol_p1).std()*np.sqrt(252), 4)

In [ ]:
fig = make_subplots(rows = 1, cols = 1,
                    shared_xaxes = True,
                    vertical_spacing = 0.08)

fig.add_trace(go.Scatter(x = df1.index, y = df1["Vol"]*100,
                         name = "Vol", line = dict(color = "blue")),
              row = 1, col = 1)

fig.update_layout(height = 600, width = 800,
                  title_text = "Annualized Volatility = " + ticker1 + " - OSM by Dalton",
                  font_color = "blue",
                  title_font_color = "black",
                  xaxis_title = "years",
                  yaxis_title = "Volatility (%)",
                  legend_title = "Vol",
                  font = dict(size = 15, color = "black"))

fig.update_layout(hovermode = "x")

# código para excluir os espaços vazios do gráfico

dt_all = pd.date_range(start = df1.index[0],
                       end = df1.index[-1],
                       freq = "D")
dt_all_py = [d.to_pydatetime() for d in dt_all]
dt_obs_py = [d.to_pydatetime() for d in df1.index]

dt_breaks = [d for d in dt_all_py if d not in dt_obs_py]

fig.update_xaxes(
    rangebreaks = [dict(values = dt_breaks)]
)
fig.show()


In [ ]:
year = "2019"

Upper_Band_12m1d =  df1[year]["Vol"][-1]*df1[year]["Adj Close"][-1] + df1[year]["Adj Close"][-1]
Lower_Band_12m1d =  df1[year]["Adj Close"][-1] - df1[year]["Vol"][-1]*df1[year]["Adj Close"][-1]

Upper_Band_12m2d =  2*df1[year]["Vol"][-1]*df1[year]["Adj Close"][-1] + df1[year]["Adj Close"][-1]
Lower_Band_12m2d =  df1[year]["Adj Close"][-1] - 2*df1[year]["Vol"][-1]*df1[year]["Adj Close"][-1]

# Annual S&D Volatility Zones chart

fig = make_subplots(rows = 1, cols = 1,
                    shared_xaxes = True,
                    vertical_spacing = 0.08)

fig.add_trace(go.Candlestick(x = df1[str(int(year)+1)].index
                             , open = df1[str(int(year)+1)]["Adj Open"], high = df1[str(int(year)+1)]["Adj High"]
                             , low = df1[str(int(year)+1)]["Adj Low"], close = df1[str(int(year)+1)]["Adj Close"]
                             , name = "Candle"
                             , increasing_line_color = "black", decreasing_line_color = "red")
              , row = 1, col = 1
             )

fig.add_hline(y = Upper_Band_12m1d, line_width = 1, line_dash = "dash", line_color = "green")
fig.add_hline(y = Lower_Band_12m1d, line_width = 1, line_dash = "dash", line_color = "red")

fig.add_hline(y = Upper_Band_12m2d, line_width = 1, line_dash = "dash", line_color = "green")
fig.add_hline(y = Lower_Band_12m2d, line_width = 1, line_dash = "dash", line_color = "red")


fig.update_layout(height = 600, width = 800
                  , title_text = "Annual S&D Volatility Zones: " + str(int(year)+1) + " " + ticker1
                  , font_color = "blue"
                  , title_font_color = "black"
                  , xaxis_title = "Years"
                  , yaxis_title = "Close"
                  , legend_title = "Vol"
                  , font = dict(size = 15, color = "Black")
                  , xaxis_rangeslider_visible = False
                 )
fig.update_layout(hovermode = "x")

# Code to exclude empty dates from the chart
dt_all = pd.date_range(start = df1.index[0]
                       , end = df1.index[-1]
                       , freq = "D")
dt_all_py = [d.to_pydatetime() for d in dt_all]
dt_obs_py = [d.to_pydatetime() for d in df1.index]

dt_breaks = [d for d in dt_all_py if d not in dt_obs_py]

fig.update_xaxes(
    rangebreaks = [dict(values = dt_breaks)]
)


fig.show()

In [ ]:
year = "2020"

Upper_Band_12m1d =  df1[year]["Vol"][-1]*df1[year]["Adj Close"][-1] + df1[year]["Adj Close"][-1]
Lower_Band_12m1d =  df1[year]["Adj Close"][-1] - df1[year]["Vol"][-1]*df1[year]["Adj Close"][-1]

Upper_Band_12m2d =  2*df1[year]["Vol"][-1]*df1[year]["Adj Close"][-1] + df1[year]["Adj Close"][-1]
Lower_Band_12m2d =  df1[year]["Adj Close"][-1] - 2*df1[year]["Vol"][-1]*df1[year]["Adj Close"][-1]

Upper_Band_12m3d =  3*df1[year]["Vol"][-1]*df1[year]["Adj Close"][-1] + df1[year]["Adj Close"][-1]

# Annual S&D Volatility Zones chart

fig = make_subplots(rows = 1, cols = 1,
                    shared_xaxes = True,
                    vertical_spacing = 0.08)

fig.add_trace(go.Candlestick(x = df1[str(int(year)+1)].index
                             , open = df1[str(int(year)+1)]["Adj Open"], high = df1[str(int(year)+1)]["Adj High"]
                             , low = df1[str(int(year)+1)]["Adj Low"], close = df1[str(int(year)+1)]["Adj Close"]
                             , name = "Candle"
                             , increasing_line_color = "black", decreasing_line_color = "red")
              , row = 1, col = 1
             )

fig.add_hline(y = Upper_Band_12m1d, line_width = 1, line_dash = "dash", line_color = "green")
fig.add_hline(y = Lower_Band_12m1d, line_width = 1, line_dash = "dash", line_color = "red")

fig.add_hline(y = Upper_Band_12m2d, line_width = 1, line_dash = "dash", line_color = "green")
fig.add_hline(y = Lower_Band_12m2d, line_width = 1, line_dash = "dash", line_color = "red")

fig.add_hline(y = Upper_Band_12m3d, line_width = 2, line_dash = "dash", line_color = "green")


fig.update_layout(height = 600, width = 800
                  , title_text = "Annual S&D Volatility Zones: " + str(int(year)+1) + " " + ticker1
                  , font_color = "blue"
                  , title_font_color = "black"
                  , xaxis_title = "Years"
                  , yaxis_title = "Close"
                  , legend_title = "Vol"
                  , font = dict(size = 15, color = "Black")
                  , xaxis_rangeslider_visible = False
                 )
fig.update_layout(hovermode = "x")

# Code to exclude empty dates from the chart
dt_all = pd.date_range(start = df1.index[0]
                       , end = df1.index[-1]
                       , freq = "D")
dt_all_py = [d.to_pydatetime() for d in dt_all]
dt_obs_py = [d.to_pydatetime() for d in df1.index]

dt_breaks = [d for d in dt_all_py if d not in dt_obs_py]

fig.update_xaxes(
    rangebreaks = [dict(values = dt_breaks)]
)


fig.show()

In [ ]:
ticker1 = "^BVSP"
df1 = yf.download(ticker1, "2012-01-01", "2022-12-31")
df1["Returns"] =  df1["Adj Close"].pct_change(1)
df1["Adj Low"] = df1["Low"] - (df1["Close"]-df1["Adj Close"])
df1["Adj High"] = df1["High"] - (df1["Close"]-df1["Adj Close"])
df1["Adj Open"] = df1["Open"] - (df1["Close"]-df1["Adj Close"])
df1["Target"] = df1["Returns"].shift(-1)
df1.tail()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,Returns,Adj Low,Adj High,Adj Open,Target
Date,,,,,,,,,,,
2022-11-11,109775.0,113010.0,109408.0,111820.0,111820.0,25038700,0.018629,109408.0,113010.0,109775.0,0.011992
2022-11-14,112256.0,114322.0,111930.0,113161.0,113161.0,15080400,0.011992,111930.0,114322.0,112256.0,-0.025786
2022-11-16,113166.0,113473.0,109512.0,110243.0,110243.0,18351000,-0.025786,109512.0,113473.0,113166.0,-0.004898
2022-11-17,110241.0,110242.0,107245.0,109703.0,109703.0,23119600,-0.004898,107245.0,110242.0,110241.0,-0.007502
2022-11-18,109706.0,111585.0,108512.0,108880.0,108880.0,18769100,-0.007502,108512.0,111585.0,109706.0,NaN


In [ ]:
vol_p1 = 20
df1["Vol"] = np.round(df1["Returns"].rolling(vol_p1).std()*np.sqrt(252), 4)

In [ ]:
fig = make_subplots(rows = 1, cols = 1,
                    shared_xaxes = True,
                    vertical_spacing = 0.08)

fig.add_trace(go.Scatter(x = df1.index, y = df1["Vol"]*100,
                         name = "Vol", line = dict(color = "blue")),
              row = 1, col = 1)

fig.update_layout(height = 600, width = 800,
                  title_text = "Annualized Volatility = " + ticker1 + " - OSM by Dalton",
                  font_color = "blue",
                  title_font_color = "black",
                  xaxis_title = "years",
                  yaxis_title = "Volatility (%)",
                  legend_title = "Vol",
                  font = dict(size = 15, color = "black"))

fig.update_layout(hovermode = "x")

# código para excluir os espaços vazios do gráfico

dt_all = pd.date_range(start = df1.index[0],
                       end = df1.index[-1],
                       freq = "D")
dt_all_py = [d.to_pydatetime() for d in dt_all]
dt_obs_py = [d.to_pydatetime() for d in df1.index]

dt_breaks = [d for d in dt_all_py if d not in dt_obs_py]

fig.update_xaxes(
    rangebreaks = [dict(values = dt_breaks)]
)
fig.show()

In [ ]:
year = "2021"

Upper_Band_12m1d =  df1[year]["Vol"][-1]*df1[year]["Adj Close"][-1] + df1[year]["Adj Close"][-1]
Lower_Band_12m1d =  df1[year]["Adj Close"][-1] - df1[year]["Vol"][-1]*df1[year]["Adj Close"][-1]

Upper_Band_12m2d =  2*df1[year]["Vol"][-1]*df1[year]["Adj Close"][-1] + df1[year]["Adj Close"][-1]
Lower_Band_12m2d =  df1[year]["Adj Close"][-1] - 2*df1[year]["Vol"][-1]*df1[year]["Adj Close"][-1]

# Annual S&D Volatility Zones chart

fig = make_subplots(rows = 1, cols = 1,
                    shared_xaxes = True,
                    vertical_spacing = 0.08)

fig.add_trace(go.Candlestick(x = df1[str(int(year)+1)].index
                             , open = df1[str(int(year)+1)]["Adj Open"], high = df1[str(int(year)+1)]["Adj High"]
                             , low = df1[str(int(year)+1)]["Adj Low"], close = df1[str(int(year)+1)]["Adj Close"]
                             , name = "Candle"
                             , increasing_line_color = "black", decreasing_line_color = "red")
              , row = 1, col = 1
             )

fig.add_hline(y = Upper_Band_12m1d, line_width = 1, line_dash = "dash", line_color = "green")
fig.add_hline(y = Lower_Band_12m1d, line_width = 1, line_dash = "dash", line_color = "red")

fig.add_hline(y = Upper_Band_12m2d, line_width = 1, line_dash = "dash", line_color = "green")
fig.add_hline(y = Lower_Band_12m2d, line_width = 1, line_dash = "dash", line_color = "red")


fig.update_layout(height = 600, width = 800
                  , title_text = "Annual S&D Volatility Zones: " + str(int(year)+1) + " " + ticker1
                  , font_color = "blue"
                  , title_font_color = "black"
                  , xaxis_title = "Years"
                  , yaxis_title = "Close"
                  , legend_title = "Vol"
                  , font = dict(size = 15, color = "Black")
                  , xaxis_rangeslider_visible = False
                 )
fig.update_layout(hovermode = "x")

# Code to exclude empty dates from the chart
dt_all = pd.date_range(start = df1.index[0]
                       , end = df1.index[-1]
                       , freq = "D")
dt_all_py = [d.to_pydatetime() for d in dt_all]
dt_obs_py = [d.to_pydatetime() for d in df1.index]

dt_breaks = [d for d in dt_all_py if d not in dt_obs_py]

fig.update_xaxes(
    rangebreaks = [dict(values = dt_breaks)]
)


fig.show()

In [ ]:
year = "2019"

Upper_Band_12m1d =  df1[year]["Vol"][-1]*df1[year]["Adj Close"][-1] + df1[year]["Adj Close"][-1]
Lower_Band_12m1d =  df1[year]["Adj Close"][-1] - df1[year]["Vol"][-1]*df1[year]["Adj Close"][-1]

Upper_Band_12m2d =  2*df1[year]["Vol"][-1]*df1[year]["Adj Close"][-1] + df1[year]["Adj Close"][-1]
Lower_Band_12m2d =  df1[year]["Adj Close"][-1] - 2*df1[year]["Vol"][-1]*df1[year]["Adj Close"][-1]

Lower_Band_12m3d =  df1[year]["Adj Close"][-1] - 3*df1[year]["Vol"][-1]*df1[year]["Adj Close"][-1]

Lower_Band_12m4d =  df1[year]["Adj Close"][-1] - 4*df1[year]["Vol"][-1]*df1[year]["Adj Close"][-1]

# Annual S&D Volatility Zones chart

fig = make_subplots(rows = 1, cols = 1,
                    shared_xaxes = True,
                    vertical_spacing = 0.08)

fig.add_trace(go.Candlestick(x = df1[str(int(year)+1)].index
                             , open = df1[str(int(year)+1)]["Adj Open"], high = df1[str(int(year)+1)]["Adj High"]
                             , low = df1[str(int(year)+1)]["Adj Low"], close = df1[str(int(year)+1)]["Adj Close"]
                             , name = "Candle"
                             , increasing_line_color = "black", decreasing_line_color = "red")
              , row = 1, col = 1
             )

fig.add_hline(y = Upper_Band_12m1d, line_width = 1, line_dash = "dash", line_color = "green")
fig.add_hline(y = Upper_Band_12m2d, line_width = 1, line_dash = "dash", line_color = "green")


fig.add_hline(y =df1[year]["Adj Close"][-1], line_width = 1, line_dash = "dash", line_color = "gray")


fig.add_hline(y = Lower_Band_12m1d, line_width = 1, line_dash = "dash", line_color = "red")
fig.add_hline(y = Lower_Band_12m2d, line_width = 1, line_dash = "dash", line_color = "red")
fig.add_hline(y = Lower_Band_12m3d, line_width = 2, line_dash = "dash", line_color = "red")
fig.add_hline(y = Lower_Band_12m4d, line_width = 3, line_dash = "dash", line_color = "red")

fig.update_layout(height = 600, width = 800
                  , title_text = "Annual S&D Volatility Zones: " + str(int(year)+1) + " " + ticker1
                  , font_color = "blue"
                  , title_font_color = "black"
                  , xaxis_title = "Years"
                  , yaxis_title = "Close"
                  , legend_title = "Vol"
                  , font = dict(size = 15, color = "Black")
                  , xaxis_rangeslider_visible = False
                 )
fig.update_layout(hovermode = "x")

# Code to exclude empty dates from the chart
dt_all = pd.date_range(start = df1.index[0]
                       , end = df1.index[-1]
                       , freq = "D")
dt_all_py = [d.to_pydatetime() for d in dt_all]
dt_obs_py = [d.to_pydatetime() for d in df1.index]

dt_breaks = [d for d in dt_all_py if d not in dt_obs_py]

fig.update_xaxes(
    rangebreaks = [dict(values = dt_breaks)]
)


fig.show()

In [ ]:
ticker1 = "BTC-USD"
df1 = yf.download(ticker1, "2012-01-01", "2022-12-31")
df1["Returns"] =  df1["Adj Close"].pct_change(1)
df1["Adj Low"] = df1["Low"] - (df1["Close"]-df1["Adj Close"])
df1["Adj High"] = df1["High"] - (df1["Close"]-df1["Adj Close"])
df1["Adj Open"] = df1["Open"] - (df1["Close"]-df1["Adj Close"])
df1["Target"] = df1["Returns"].shift(-1)
df1.tail()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,Returns,Adj Low,Adj High,Adj Open,Target
Date,,,,,,,,,,,
2022-11-16,16884.341797,16960.294922,16430.111328,16669.439453,16669.439453,33925512989,-0.012744,16430.111328,16960.294922,16884.341797,0.001085
2022-11-17,16670.425781,16726.439453,16460.683594,16687.517578,16687.517578,27868914022,0.001085,16460.683594,16726.439453,16670.425781,0.000615
2022-11-18,16687.912109,16947.058594,16564.611328,16697.777344,16697.777344,26862218609,0.000615,16564.611328,16947.058594,16687.912109,0.000825
2022-11-19,16696.220703,16797.876953,16570.410156,16711.546875,16711.546875,16106223492,0.000825,16570.410156,16797.876953,16696.220703,-0.027266
2022-11-20,16711.566406,16743.855469,16255.886719,16255.886719,16255.886719,21173653504,-0.027266,16255.886719,16743.855469,16711.566406,NaN


In [ ]:
vol_p1 = 20
df1["Vol"] = np.round(df1["Returns"].rolling(vol_p1).std()*np.sqrt(252), 4)

In [ ]:
year = "2019"

Upper_Band_12m1d =  df1[year]["Vol"][-1]*df1[year]["Adj Close"][-1] + df1[year]["Adj Close"][-1]
Upper_Band_12m2d =  2*df1[year]["Vol"][-1]*df1[year]["Adj Close"][-1] + df1[year]["Adj Close"][-1]
Upper_Band_12m3d =  3*df1[year]["Vol"][-1]*df1[year]["Adj Close"][-1] + df1[year]["Adj Close"][-1]
Upper_Band_12m4d =  4*df1[year]["Vol"][-1]*df1[year]["Adj Close"][-1] + df1[year]["Adj Close"][-1]
Upper_Band_12m5d =  5*df1[year]["Vol"][-1]*df1[year]["Adj Close"][-1] + df1[year]["Adj Close"][-1]
Upper_Band_12m6d =  6*df1[year]["Vol"][-1]*df1[year]["Adj Close"][-1] + df1[year]["Adj Close"][-1]


Lower_Band_12m1d =  df1[year]["Adj Close"][-1] - df1[year]["Vol"][-1]*df1[year]["Adj Close"][-1]
Lower_Band_12m2d =  df1[year]["Adj Close"][-1] - 2*df1[year]["Vol"][-1]*df1[year]["Adj Close"][-1]
Lower_Band_12m3d =  df1[year]["Adj Close"][-1] - 3*df1[year]["Vol"][-1]*df1[year]["Adj Close"][-1]
Lower_Band_12m4d =  df1[year]["Adj Close"][-1] - 4*df1[year]["Vol"][-1]*df1[year]["Adj Close"][-1]

# Annual S&D Volatility Zones chart

fig = make_subplots(rows = 1, cols = 1,
                    shared_xaxes = True,
                    vertical_spacing = 0.08)

fig.add_trace(go.Candlestick(x = df1[str(int(year)+1)].index
                             , open = df1[str(int(year)+1)]["Adj Open"], high = df1[str(int(year)+1)]["Adj High"]
                             , low = df1[str(int(year)+1)]["Adj Low"], close = df1[str(int(year)+1)]["Adj Close"]
                             , name = "Candle"
                             , increasing_line_color = "black", decreasing_line_color = "red")
              , row = 1, col = 1
             )

fig.add_hline(y = Upper_Band_12m1d, line_width = 1, line_dash = "dash", line_color = "green")
fig.add_hline(y = Upper_Band_12m2d, line_width = 1, line_dash = "dash", line_color = "green")
fig.add_hline(y = Upper_Band_12m3d, line_width = 2, line_dash = "dash", line_color = "green")
fig.add_hline(y = Upper_Band_12m4d, line_width = 3, line_dash = "dash", line_color = "green")
fig.add_hline(y = Upper_Band_12m5d, line_width = 3, line_dash = "dash", line_color = "green")
fig.add_hline(y = Upper_Band_12m6d, line_width = 4, line_dash = "dash", line_color = "green")

fig.add_hline(y =df1[year]["Adj Close"][-1], line_width = 1, line_dash = "dash", line_color = "gray")

fig.add_hline(y = Lower_Band_12m1d, line_width = 1, line_dash = "dash", line_color = "red")
fig.add_hline(y = Lower_Band_12m2d, line_width = 1, line_dash = "dash", line_color = "red")
#fig.add_hline(y = Lower_Band_12m3d, line_width = 1, line_dash = "dash", line_color = "red")
#fig.add_hline(y = Lower_Band_12m4d, line_width = 1, line_dash = "dash", line_color = "red")


fig.update_layout(height = 600, width = 800
                  , title_text = "Annual S&D Volatility Zones: " + str(int(year)+1) + " " + ticker1 + " - OSM by Dalton"
                  , font_color = "blue"
                  , title_font_color = "black"
                  , xaxis_title = "Years"
                  , yaxis_title = "Close"
                  , legend_title = "Vol"
                  , font = dict(size = 15, color = "Black")
                  , xaxis_rangeslider_visible = False
                 )
fig.update_layout(hovermode = "x")

# Code to exclude empty dates from the chart
dt_all = pd.date_range(start = df1.index[0]
                       , end = df1.index[-1]
                       , freq = "D")
dt_all_py = [d.to_pydatetime() for d in dt_all]
dt_obs_py = [d.to_pydatetime() for d in df1.index]

dt_breaks = [d for d in dt_all_py if d not in dt_obs_py]

fig.update_xaxes(
    rangebreaks = [dict(values = dt_breaks)]
)


fig.show()